https://medium.com/@yulongtsai/datalab-bigquery-python-kkbox-churn-prediction-f2a7245c5d99

In [1]:
# REF: https://medium.com/@yulongtsai/kaggle-kkbox-churn-prediction-top5-c0ea4c9b3f1a

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt 

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

* v1吃
* v2吃4.5G RAM

In [3]:
df_logv1 = pd.read_csv("~/data/kkbox/user_logs.csv")
df_logv1.info()

MemoryError: 

In [2]:
# load data
# df_logv1 = pd.read_csv("~/data/kkbox/user_logs.csv")
df_logv2 = pd.read_csv("~/data/kkbox/user_logs_v2.csv")
df_logv2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18396362 entries, 0 to 18396361
Data columns (total 9 columns):
msno          object
date          int64
num_25        int64
num_50        int64
num_75        int64
num_985       int64
num_100       int64
num_unq       int64
total_secs    float64
dtypes: float64(1), int64(7), object(1)
memory usage: 1.2+ GB


batch load 比較安全 (但還是會吃超過60G)

In [9]:
dict(zip(df_cols[2:8], ['int8' for _ in range(6)]))

{'num_25': 'int8',
 'num_50': 'int8',
 'num_75': 'int8',
 'num_985': 'int8',
 'num_100': 'int8',
 'num_unq': 'int8'}

In [2]:
%%time
# 1000萬筆一次
chunksize = 10 ** 7
filename = '~/data/kkbox/user_logs.csv'
df_buffer = []
df_concat = []
df_cols = []
count = 0
for chunk in pd.read_csv(filename, chunksize=chunksize):
    if count > 30:
        df_buffer = []
        break
    if count % 10 == 0:
        df_concat.append(pd.DataFrame(df_buffer, columns=df_cols))
        df_buffer = []
        
    if not df_cols:
        df_cols = chunk.columns.values.tolist()
    df_buffer.extend(chunk.values.tolist())
    count +=1
    # 能省則省
    del chunk

df_logv1 = pd.concat(df_concat)
del df_concat
df_logv1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300000000 entries, 0 to 99999999
Data columns (total 9 columns):
msno          object
date          int64
num_25        int64
num_50        int64
num_75        int64
num_985       int64
num_100       int64
num_unq       int64
total_secs    float64
dtypes: float64(1), int64(7), object(1)
memory usage: 22.4+ GB
CPU times: user 12min 50s, sys: 1min 34s, total: 14min 24s
Wall time: 12min 14s


轉type以減少memory

In [3]:
type_dict = dict(zip(df_cols[2:8], ['int8' for _ in range(6)]))
df_logv1 = df_logv1.astype(type_dict)
df_logv1.info()
# df_log = df_logv1.astype(type_dict)
# del df_logv1
# df_log.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300000000 entries, 0 to 99999999
Data columns (total 9 columns):
msno          object
date          int64
num_25        int8
num_50        int8
num_75        int8
num_985       int8
num_100       int8
num_unq       int8
total_secs    float64
dtypes: float64(1), int64(1), int8(6), object(1)
memory usage: 10.6+ GB


**計算2億筆的NA會額外吃13G RAM**

**計算3億筆的NA會額外吃22G RAM**

In [3]:
df_logv1.isna().sum()

msno          0
date          0
num_25        0
num_50        0
num_75        0
num_985       0
num_100       0
num_unq       0
total_secs    0
dtype: int64

In [4]:
df_logv1.shape

(300000000, 9)

In [8]:
pd.options.display.float_format = '{:.0f}'.format

In [9]:
%%time
df_logv1.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
count,3e+08,3e+08,3e+08,3e+08,3e+08,3e+08,3e+08,3e+08
mean,2.01574e+07,6.51117,1.64024,1.01705,1.12867,30.7227,30.091,-1.44121e+12
std,6180.79,14.5716,4.24669,2.1777,2.86089,42.2928,33.7522,1.15404e+14
min,2.01501e+07,0,0,0,0,0,1,-9.22337e+15
25%,2.01508e+07,0,0,0,0,6,8,1891.57
50%,2.01603e+07,2,1,0,0,17,19,4628.57
75%,2.01609e+07,7,2,1,1,38,40,10262.3
max,2.01702e+07,18798,1710,1690,2747,34696,4784,9.22337e+15


In [2]:
# combine 2 user_logs

In [4]:
# to pkl
df_logv1.to_pickle("./pkl_files/log300000000.pkl")

### 製作新特徵

In [ ]:
# 列出資料的時間範圍

In [ ]:
# 計算每日每個使用者的總聽歌數

In [ ]:
# 計算聽幾天